In [1]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade typing_extensions

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
import gradio as gr

In [4]:
# Load the dataset
df = pd.read_csv('Resources/SMSSpamCollection.csv', sep='\t', names=['label', 'text'])

In [5]:
# Drop rows with missing values in the 'text' column
df.dropna(subset=['text'], inplace=True)

# Function to classify SMS
def sms_classification(df):
    # Set the features and target variable
    features = df['text']
    target = df['label']
    
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.33, random_state=42, stratify=target)
    
    # Build the pipeline
    text_clf = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', MultinomialNB())
    ])
    
    # Fit the model
    text_clf.fit(X_train, y_train)
    
    # Evaluate the model
    y_pred = text_clf.predict(X_test)
    print(classification_report(y_test, y_pred))
    print("Accuracy:", accuracy_score(y_test, y_pred))
    
    return text_clf

# Train the model
text_clf = sms_classification(df)

# Function to predict SMS classification
def sms_prediction(text):
    # Make a prediction
    prediction = text_clf.predict([text])[0]
    
    # Determine if the text is ham or spam
    if prediction == 'ham':
        return f'The text message: "{text}", is not spam.'
    else:
        return f'The text message: "{text}", is spam.'

# Create Gradio interface
def classify_sms(text):
    return sms_prediction(text)

iface = gr.Interface(
    fn=classify_sms,
    inputs=gr.Textbox(lines=2, placeholder='Enter SMS text here...'),
    outputs='text',
    title='SMS Spam Classifier',
    description='Enter an SMS message to classify it as spam or not spam.'
)

# Launch the application
iface.launch(share=True)

              precision    recall  f1-score   support

         ham       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

Accuracy: 1.0
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://86860ff623c76a2eee.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
